# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

In [15]:
from sframe import SFrame
from sframe import SArray

In [9]:
sales = SFrame('kc_house_data.gl/')
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [10]:
train_data, test_data = sales.random_split(0.8,seed=0)

# Test on some aggerate functions

In [12]:
prices = sales['price']
sum_prices = prices.sum()
num_houses = prices.size()
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean()
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


In [13]:
half_prices = 0.5*prices
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum()
print 'the sum of price squared is: ' + str(sum_prices_squared)

the sum of price squared is: 9.21732513355e+15


# Build a generic simple linear regression function

In [17]:
def simple_linear_regression(input_feature, output):
    feature_sum = input_feature.sum()
    output_sum = output.sum()
    sum_product_input_and_output = (input_feature*output).sum()
    squared_input = (input_feature*input_feature).sum()
    N = input_feature.size()
    slope = (sum_product_input_and_output - feature_sum * output_sum / N) / (squared_input - feature_sum*feature_sum / N)
    intercept = output_sum / N - slope * feature_sum / N
    return (intercept, slope)

Testing for the function

In [18]:
test_feature = SArray(range(5))
test_output = SArray(1 + 1*test_feature)
(test_intercept, test_slope) = simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1
Slope: 1


In [19]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])
print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0765749
Slope: 281.958838568


# Predicting Values

In [20]:
def get_regression_predictions(input_feature, intercept, slope):
    predicted_values = input_feature * slope + intercept
    return predicted_values

In [22]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


In [26]:
# Calculate RSS of the model
def get_residual_sum_of_squares(input_feature, test_output, test_intercept, test_slope):
    predictions = get_regression_predictions(input_feature,test_intercept, test_slope)
    RSS = ((test_output - predictions) * (test_output - predictions)).sum()
    return RSS

In [27]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)

0


In [28]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print "The RSS of predicting Prices based on Square Feet is : " + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 1.20191835632e+15


# Predict the squarefeet given price

In [29]:
def inverse_regression_predictions(output, intercept, slope):
    estimated_feature = (output - intercept) / slope
    return estimated_feature

In [30]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


# Estimate prices from bedrooms

In [36]:
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

In [38]:
rss_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedrooms_intercept, bedrooms_slope)
rss_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print "The RSS of predicting Prices based on Square Feet is : " + str(rss_sqft)
print "The RSS of predicting Prices based on Bedrooms is : " + str(rss_bedrooms)

The RSS of predicting Prices based on Square Feet is : 2.75402936247e+14
The RSS of predicting Prices based on Bedrooms is : 4.93364582868e+14
